Limpieza de datos Rooms

In [569]:
import pandas as pd
import numpy as np
import re 

In [570]:
data_location = "./Data/properati.csv"
data = pd.read_csv(data_location, sep=",")
print(data.dtypes)
data.head(3)

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [571]:
data["rooms_completo"] =data["rooms"]
data_room_null_description = data.description.astype(str).str.lower()
data_room_null_title = data.title.astype(str).str.lower()
display(data_room_null_description)
display(data_room_null_title)

0         2 ambientes tipo casa planta baja por pasillo,...
1         venta de departamento en décimo piso al frente...
2         2 ambientes 3er piso lateral living comedor am...
3         ph 3 ambientes con patio. hay 3 deptos en lote...
4         departamento con fantástica iluminación natura...
                                ...                        
121215    torre forum alcorta - máxima categoría.impecab...
121216    excelente e impecable casa en venta en las lom...
121217    venta departamento ambiente divisible a estren...
121218    2 amb al contrafrente,  luminoso. el departame...
121219    departamento de 77 m2 apto profesional. edific...
Name: description, Length: 121220, dtype: object

0                        2 amb tipo casa sin expensas en pb
1         venta depto 2 dorm. a estrenar 7 e/ 36 y 37   ...
2                  2 amb 3er piso con ascensor apto credito
3                              ph 3 amb. cfte. reciclado   
4         depto 2 amb al contrafrente zona centro/plaza ...
                                ...                        
121215           torre forum alcorta- impecable 3 ambientes
121216    ruca inmuebles | venta | lomas de san isidro |...
121217    venta departamento ambiente divisible a estren...
121218    2 amb. c/ dep. de servicio al contrafrente| re...
121219           departamento apto profesional en el centro
Name: title, Length: 121220, dtype: object

In [572]:
print("Primer grupo de filas sin rooms")
data["rooms_completo"].isnull().value_counts()

Primer grupo de filas sin rooms


True     73830
False    47390
Name: rooms_completo, dtype: int64

Seteamos los que son monoambientes
Primero buscamos las palabras que identifiquen al monoambiente. 
Luego buscamos las coincidencias dentro del universo de coincidencias del campo descripción
donde hubo coincidencias, le asignamos 1. 

In [573]:
def fun_buscar_patt(data_validar,patt_room):
    patt_regex = re.compile(patt_room)
    data_match = data_validar.apply(lambda x:  x if x is np.NaN else patt_regex.search(x))
    return data_match

In [574]:
def fun_buscar_disminuir(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    data.loc[room_null_mask,"rooms_completo"]= encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar)).astype(int)
    print("Disminución en función a los ambientes tipo:",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [575]:
def fun_buscar_disminuir_mapper(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna y se mapea el texto###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    numcol_cat_mapper = {
       'un': 1,'uun':1, 'uno': 1,'dos': 2,'tres': 3,'cuatro':4,
        'cinco':5,'sinco':5,'seis':6,'ceis':6,'siete':7,
        'ciete':7,'ocho':8,'nueve':9,'diez':10,'once':11,
        'doce':12,'trece':13,'catorce':14,'quince':15,
        'dieciseis':16,'diecisiete':17,'dieciocho':18,'diecinueve':19,
        'veinte':20
    }
    encontradas1 = encontradas.map(numcol_cat_mapper)
    data.loc[room_null_mask,"rooms_completo"]= encontradas1[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    print("Disminución en función a los ambientes :",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [576]:
#validar que luego elija siempre la última que encuentre

In [577]:
def fun_buscar_disminuir_mapper_2(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna y se mapea el texto###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    numcol_cat_mapper = {
                'segundo': 2,'tercer':3, 'cuarto': 4,'quinto': 5,'sexto': 6
    }
    encontradas1 = encontradas.map(numcol_cat_mapper)
    data.loc[room_null_mask,"rooms_completo"]= encontradas1[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    print("Disminución en función a los ambientes :",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [578]:
def disminuciones_repeticion(serie,patt):
    print(data["rooms_completo"].isnull().value_counts())
    data_encontrada = serie.apply(lambda x: x if x is np.NaN else x.count(patt))
    maskarita= data_encontrada>0|data["rooms_completo"].isnull()
    data.loc[maskarita,"rooms_completo"] = data_encontrada[maskarita]
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [579]:
data_room_match = fun_buscar_patt(data_room_null_description,r"(mono ambiente|monoambiente|mono-ambiente|Monoamb)")
data_room_match_mask = data_room_match.notnull()
data_room_match[data_room_match_mask]=0
data_room_mask = data["rooms_completo"].isnull()

data.loc[data_room_match_mask,"rooms_completo"] = data_room_match[data_room_match_mask].apply(lambda x: x if x is np.NaN else x)
print("Primera disminución Monoambientes")
display(data["rooms_completo"].isnull().value_counts())

Primera disminución Monoambientes


True     69080
False    52140
Name: rooms_completo, dtype: int64

In [580]:
data_room_match = fun_buscar_patt(data_room_null_title,r"(mono ambiente|monoambiente|mono-ambiente|monoamb)")
data_room_match_mask = data_room_match.notnull()
data_room_match[data_room_match_mask]=0
data_room_mask = data["rooms_completo"].isnull()

data.loc[data_room_match_mask,"rooms_completo"] = data_room_match[data_room_match_mask].apply(lambda x: x if x is np.NaN else x)
print("Primera disminución Monoambientes")
display(data["rooms_completo"].isnull().value_counts())

Primera disminución Monoambientes


True     68675
False    52545
Name: rooms_completo, dtype: int64

In [581]:
resultado = fun_buscar_disminuir(data_room_null_description,r"[\s](\d\d)[\s]?habitaci",0)
print(resultado)

Disminución en función a los ambientes tipo: [\s](\d\d)[\s]?habitaci
True     68631
False    52589
Name: rooms_completo, dtype: int64


In [582]:
resultado = fun_buscar_disminuir_mapper(data_room_null_description,r"(\w+)[\s]?(dormitorio|domitorio)",0)
print(resultado)

Disminución en función a los ambientes : (\w+)[\s]?(dormitorio|domitorio)
False    64513
True     56707
Name: rooms_completo, dtype: int64


In [583]:
resultado = fun_buscar_disminuir_mapper_2(data_room_null_description,r"(\w+)[\s]?(dormitorio|domitorio)",0)
print(resultado)

Disminución en función a los ambientes : (\w+)[\s]?(dormitorio|domitorio)
False    65214
True     56006
Name: rooms_completo, dtype: int64


In [584]:
resultado = fun_buscar_disminuir_mapper(data_room_null_description,r"(\w+)[\s]?(ambiente)",1)
print(resultado)

Disminución en función a los ambientes : (\w+)[\s]?(ambiente)
False    67069
True     54151
Name: rooms_completo, dtype: int64


In [585]:
resultado = fun_buscar_disminuir(data_room_null_description,r"[\s](\d)[\s]?(amb)",1)
print(resultado)

Disminución en función a los ambientes tipo: [\s](\d)[\s]?(amb)
False    82384
True     38836
Name: rooms_completo, dtype: int64


In [586]:
resultado = fun_buscar_disminuir(data_room_null_description,r"\D(\d)[\s]?(dorm)",0)
print(resultado)

Disminución en función a los ambientes tipo: \D(\d)[\s]?(dorm)
False    100893
True      20327
Name: rooms_completo, dtype: int64


In [587]:
resultado = fun_buscar_disminuir(data_room_null_title,r"\D(\d?\d)[\s]?(dorm)",0)
print(resultado)

Disminución en función a los ambientes tipo: \D(\d?\d)[\s]?(dorm)
False    101365
True      19855
Name: rooms_completo, dtype: int64


In [588]:
resultado = fun_buscar_disminuir(data_room_null_description,r"(\d)[\s]?(suite)",0)
print(resultado)

Disminución en función a los ambientes tipo: (\d)[\s]?(suite)
False    101515
True      19705
Name: rooms_completo, dtype: int64


Buscamos ahora las restantes en el título

In [589]:
resultado = fun_buscar_disminuir(data_room_null_title,r"([\s]\d?\d)[\s]?amb",1)
print(resultado)

Disminución en función a los ambientes tipo: ([\s]\d?\d)[\s]?amb
False    102495
True      18725
Name: rooms_completo, dtype: int64


In [590]:
resultado = fun_buscar_disminuir_mapper(data_room_null_title,r"(\w+)[\s]?(amb)",1)
print(resultado)

Disminución en función a los ambientes : (\w+)[\s]?(amb)
False    102663
True      18557
Name: rooms_completo, dtype: int64


In [591]:
disminuciones_repeticion(data_room_null_description,"dormitorio")

False    102663
True      18557
Name: rooms_completo, dtype: int64


False    104570
True      16650
Name: rooms_completo, dtype: int64

In [592]:
disminuciones_repeticion(data_room_null_description,"habitacion")


False    104570
True      16650
Name: rooms_completo, dtype: int64


False    104948
True      16272
Name: rooms_completo, dtype: int64

In [593]:
disminuciones_repeticion(data_room_null_description,"suite")

False    104948
True      16272
Name: rooms_completo, dtype: int64


False    105281
True      15939
Name: rooms_completo, dtype: int64

## `SETEANDO RESTO DE ROOMS EN NULL`
ALGUNOS AJUSTES POSTERIORES ESPECÍFICOS PORQUE LA REGEX LOS TOMÓ Y NO APLICABA


AQUÍ SETEO EL RESTO QUE SE ENCUENTRA EN NULL

In [594]:
def fun_back(idreg):
    data.loc[idreg:idreg,"rooms_completo"]=data.loc[idreg:idreg,"rooms"]
    return data.loc[idreg:idreg,"rooms_completo"]
fun_back(53793)
fun_back(23884)

23884   NaN
Name: rooms_completo, dtype: float64

In [595]:
data["rooms_completo"].fillna(0,inplace = True) 
print(data["rooms_completo"].isnull().value_counts())

False    121220
Name: rooms_completo, dtype: int64


## `AGREGANDO CAMPOS NUEVOS`
Se crean nuevos campos para definir características que pueden revalorizar la propiedad
____

In [596]:
data_encontrada1= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(pileta|piscina|pisina|picina)")
encontradas_pileta = data_encontrada1[data_encontrada1.notnull()].apply(lambda x: x.group(0))
data["PILETA"] = encontradas_pileta.apply(lambda x: False if x.count("sin")>0 else True)
data["PILETA"].fillna(value=False, inplace=True)
data["PILETA"].value_counts()

False    104706
True      16514
Name: PILETA, dtype: int64

In [597]:
data_encontrada_parrilla= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(parrilla|asado|grill|churrasquera)")
encontradas_parrilla = data_encontrada_parrilla[data_encontrada_parrilla.notnull()].apply(lambda x: x.group(0))
data["PARRILLA"] = encontradas_parrilla.apply(lambda x: False if x.count("sin")>0 else True)
data["PARRILLA"].fillna(value=False, inplace=True)
# data["PARRILLA"]= data_encontrada_parrilla.notnull()
data["PARRILLA"].value_counts()

False    93220
True     28000
Name: PARRILLA, dtype: int64

In [598]:
data_encontrada_patio= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(patio|jardin)")
encontradas_patio = data_encontrada_patio[data_encontrada_patio.notnull()].apply(lambda x: x.group(0))
data["PATIO_JARDIN"] = encontradas_patio.apply(lambda x: False if x.count("sin")>0 else True)
data["PATIO_JARDIN"].fillna(value=False, inplace=True)
data["PATIO_JARDIN"].value_counts()

False    104289
True      16931
Name: PATIO_JARDIN, dtype: int64

ESTO ES PARA REALIZAR VALIDACIÓN DE DATOS


______


In [602]:
bins= [1,2,3,4,5,6,7,8,9,10,15,20,25,30,40]
room= pd.cut(data.rooms_completo, bins,right=False)

0                NaN
1         [2.0, 3.0)
2                NaN
3         [2.0, 3.0)
4         [1.0, 2.0)
             ...    
121215    [1.0, 2.0)
121216    [1.0, 2.0)
121217           NaN
121218           NaN
121219           NaN
Name: rooms_completo, Length: 121220, dtype: category
Categories (14, interval[int64]): [[1, 2) < [2, 3) < [3, 4) < [4, 5) ... [15, 20) < [20, 25) < [25, 30) < [30, 40)]

In [603]:
data.pivot_table('rooms', index=[room], columns='property_type', aggfunc='count' )

property_type,PH,apartment,house,store
rooms_completo,,,,
"[1, 2)",1709.0,18224.0,7872.0,67.0
"[2, 3)",885.0,6432.0,2384.0,16.0
"[3, 4)",420.0,2454.0,1355.0,9.0
"[4, 5)",137.0,762.0,583.0,4.0
"[5, 6)",42.0,221.0,281.0,3.0
"[6, 7)",18.0,101.0,104.0,2.0
"[7, 8)",5.0,71.0,58.0,3.0
"[8, 9)",1.0,52.0,29.0,0.0
"[9, 10)",NaN,20.0,15.0,NaN
